In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features.build_features import obtener_diccionario_mai

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
codigos_mai = obtener_diccionario_mai()

## 1. Análisis de procedimientos sin código MAI

En este cuadernillo se quiere realizar un control de calidad sobre los códigos ingresados
para cada procedimiento efectuado. En específico, se quieren responder las siguientes preguntas:

1. ¿Hay códigos que estén ausentes en el arancel MAI?
2. De los 1, ¿Cuál es el nombre de la operación ingresada manualmente?
3. De los 1, ¿Existen distintas glosas ingresadas para un mismo código?

In [ ]:
df = pd.read_csv("../data/processed/df_procesada.csv", encoding="latin-1", sep=";")
df = df.merge(codigos_mai, how="left", left_on="codigo_i", right_on="CODIGO")